# Рекомендательные системы

### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k**.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - **id**-шниками просмотренных и **id**-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: **id** просмотренных товаров через , затем идёт ; после чего следуют **id** купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4**; или **1,2,3,4;5,6**.

Гарантируется, что среди **id** купленных товаров все различные.

### Важно:

* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k / precision@k**.

### Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    * сортировка просмотренных id по популярности (частота появления в просмотренных),
    * сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

### Дополнительные вопросы

1. Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров **recall@k** меняется. Подумайте, как оценить минимальное и максимальное значение **recall@k** в зависимости от правила сортировки.
2. Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
train_data = pd.read_csv('datasets/coursera_sessions_train.txt', sep=';', header=None)
train_data.columns = ['browsed', 'purchased']

train_data.browsed = train_data.browsed.apply(lambda x: x.split(','))
train_data.purchased = train_data.purchased.apply(lambda x: x.split(',') if x is not np.nan else np.nan)
train_data.head(10)

,browsed,purchased
0,"[0, 1, 2, 3, 4, 5]",NaN
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN
2,"[16, 17, 18, 19, 20, 21]",NaN
3,"[24, 25, 26, 27, 24]",NaN
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN
5,[42],NaN
6,"[47, 48, 49]",NaN
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
8,"[71, 72, 73, 74]",NaN
9,"[76, 77, 78]",NaN


In [3]:
test_data = pd.read_csv('datasets/coursera_sessions_test.txt', sep=';', header=None)
test_data.columns = ['browsed', 'purchased']

test_data.browsed = test_data.browsed.apply(lambda x: x.split(','))
test_data.purchased = test_data.purchased.apply(lambda x: x.split(',') if x is not np.nan else np.nan)
test_data.head(10)

,browsed,purchased
0,"[6, 7, 8]",NaN
1,"[13, 14, 15]",NaN
2,"[22, 23]",NaN
3,"[28, 29, 30, 31, 32, 33]",NaN
4,"[40, 41]",NaN
5,"[43, 44, 43, 45, 43, 45, 43, 46]",NaN
6,"[50, 51, 47, 52, 49, 53, 54, 55, 56, 57, 58]",NaN
7,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]"
8,[75],NaN
9,"[79, 80, 81, 82, 83]",NaN


In [4]:
browsed_counter, purchased_counter = Counter(), Counter()
for _index, session in train_data.iterrows():
    browsed_counter.update(session.browsed)
    if session.purchased is not np.nan:
        purchased_counter.update(session.purchased)

Топ 10 просмотренных товаров:

In [5]:
for id_, freq in browsed_counter.most_common(10):
    print(f'id: {id_}, freq: {freq}')

id: 73, freq: 677
id: 158, freq: 641
id: 204, freq: 396
id: 262, freq: 387
id: 162, freq: 318
id: 7, freq: 312
id: 137, freq: 306
id: 1185, freq: 284
id: 6, freq: 283
id: 170, freq: 280


Топ 10 просмотренных по покупаемости товаров:

In [6]:
for id_, freq in purchased_counter.most_common(10):
    print(f'id: {id_}, freq: {freq}')

id: 158, freq: 14
id: 204, freq: 12
id: 3324, freq: 11
id: 73, freq: 11
id: 5569, freq: 10
id: 3149, freq: 10
id: 977, freq: 10
id: 1181, freq: 9
id: 162, freq: 8
id: 1852, freq: 7


In [7]:
def recall(row, k, counter):
    ids = pd.unique(row.browsed)
    positions = list(range(len(ids)))
    freqs = [counter[id_] for id_ in ids]
    
    brows_data = pd.DataFrame({'id_': ids, 'pos': positions, 'freq': freqs})
    brows_data.sort_values(['freq', 'pos'], ascending=[False, True], inplace=True)

    bought = len(set(brows_data.id_[:k]).intersection(row.purchased))
    
    return bought / len(row.purchased)

In [8]:
def precision(row, k, counter):
    ids = pd.unique(row.browsed)
    positions = list(range(len(ids)))
    freqs = [counter[id_] for id_ in ids]
    
    brows_data = pd.DataFrame({'id_': ids, 'pos': positions, 'freq': freqs})
    brows_data.sort_values(['freq', 'pos'], ascending=[False, True], inplace=True)

    bought = len(set(brows_data.id_[:k]).intersection(row.purchased))
    
    return bought / k

Значения метрик **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей выборке для алгоритма, основанного на частоте просмотров товаров:

In [9]:
train_data.dropna(inplace=True)

In [10]:
avg_recall_1 = round(sum(train_data.apply(lambda row: recall(row, 1, browsed_counter), axis=1)) / len(train_data), 2)
avg_recall_1

0.44

In [11]:
avg_prec_1 = round(sum(train_data.apply(lambda row: precision(row, 1, browsed_counter), axis=1)) / len(train_data), 2)
avg_prec_1

0.51

In [12]:
avg_recall_5 = round(sum(train_data.apply(lambda row: recall(row, 5, browsed_counter), axis=1)) / len(train_data), 2)
avg_recall_5

0.82

In [13]:
avg_prec_5 = round(sum(train_data.apply(lambda row: precision(row, 5, browsed_counter), axis=1)) / len(train_data), 2)
avg_prec_5

0.21

In [14]:
with open('ans1.txt', 'w') as f:
    f.write(' '.join([str(x) for x in [avg_recall_1, avg_prec_1, avg_recall_5, avg_prec_5]]))

Значения метрик **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на тестовой выборке для алгоритма, основанного на частоте просмотров товаров:

In [15]:
test_data.dropna(inplace=True)

In [16]:
avg_recall_1 = round(sum(test_data.apply(lambda row: recall(row, 1, browsed_counter), axis=1)) / len(test_data), 2)
avg_recall_1

0.42

In [17]:
avg_prec_1 = round(sum(test_data.apply(lambda row: precision(row, 1, browsed_counter), axis=1)) / len(test_data), 2)
avg_prec_1

0.48

In [18]:
avg_recall_5 = round(sum(test_data.apply(lambda row: recall(row, 5, browsed_counter), axis=1)) / len(test_data), 2)
avg_recall_5

0.8

In [19]:
avg_prec_5 = round(sum(test_data.apply(lambda row: precision(row, 5, browsed_counter), axis=1)) / len(test_data), 2)
avg_prec_5

0.2

In [20]:
with open('ans2.txt', 'w') as f:
    f.write(' '.join([str(x) for x in [avg_recall_1, avg_prec_1, avg_recall_5, avg_prec_5]]))

Значения метрик **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей выборке для алгоритма, основанного на частоте покупок товаров:

In [21]:
avg_recall_1 = round(sum(train_data.apply(lambda row: recall(row, 1, purchased_counter), axis=1)) / len(train_data), 2)
avg_recall_1

0.69

In [22]:
avg_prec_1 = round(sum(train_data.apply(lambda row: precision(row, 1, purchased_counter), axis=1)) / len(train_data), 2)
avg_prec_1

0.8

In [23]:
avg_recall_5 = round(sum(train_data.apply(lambda row: recall(row, 5, purchased_counter), axis=1)) / len(train_data), 2)
avg_recall_5

0.93

In [24]:
avg_prec_5 = round(sum(train_data.apply(lambda row: precision(row, 5, purchased_counter), axis=1)) / len(train_data), 2)
avg_prec_5

0.25

In [25]:
with open('ans3.txt', 'w') as f:
    f.write(' '.join([str(x) for x in [avg_recall_1, avg_prec_1, avg_recall_5, avg_prec_5]]))

Значения метрик **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на тестовой выборке для алгоритма, основанного на частоте покупок товаров:

In [26]:
avg_recall_1 = round(sum(test_data.apply(lambda row: recall(row, 1, purchased_counter), axis=1)) / len(test_data), 2)
avg_recall_1

0.46

In [27]:
avg_prec_1 = round(sum(test_data.apply(lambda row: precision(row, 1, purchased_counter), axis=1)) / len(test_data), 2)
avg_prec_1

0.53

In [28]:
avg_recall_5 = round(sum(test_data.apply(lambda row: recall(row, 5, purchased_counter), axis=1)) / len(test_data), 2)
avg_recall_5

0.82

In [29]:
avg_prec_5 = round(sum(test_data.apply(lambda row: precision(row, 5, purchased_counter), axis=1)) / len(test_data), 2)
avg_prec_5

0.21

In [30]:
with open('ans4.txt', 'w') as f:
    f.write(' '.join([str(x) for x in [avg_recall_1, avg_prec_1, avg_recall_5, avg_prec_5]]))